In [2]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist, fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

In [3]:
class DUALGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_dim = self.img_rows*self.img_cols

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.D_A = self.build_discriminator()
        self.D_A.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
        self.D_B = self.build_discriminator()
        self.D_B.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.G_AB = self.build_generator()
        self.G_BA = self.build_generator()

        # For the combined model we will only train the generators
        self.D_A.trainable = False
        self.D_B.trainable = False

        # The generator takes images from their respective domains as inputs
        imgs_A = Input(shape=(self.img_dim,))
        imgs_B = Input(shape=(self.img_dim,))

        # Generators translates the images to the opposite domain
        fake_B = self.G_AB(imgs_A)
        fake_A = self.G_BA(imgs_B)

        # The discriminators determines validity of translated images
        valid_A = self.D_A(fake_A)
        valid_B = self.D_B(fake_B)

        # Generators translate the images back to their original domain
        recov_A = self.G_BA(fake_B)
        recov_B = self.G_AB(fake_A)

        # The combined model  (stacked generators and discriminators)
        self.combined = Model(inputs=[imgs_A, imgs_B], outputs=[valid_A, valid_B, recov_A, recov_B])
        self.combined.compile(loss=[self.wasserstein_loss, self.wasserstein_loss, 'mae', 'mae'],
                            optimizer=optimizer,
                            loss_weights=[1, 1, 100, 100])

    def build_generator(self):

        X = Input(shape=(self.img_dim,))

        model = Sequential()
        model.add(Dense(256, input_dim=self.img_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dropout(0.4))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dropout(0.4))
        model.add(Dense(self.img_dim, activation='tanh'))

        X_translated = model(X)

        return Model(X, X_translated)

    def build_discriminator(self):

        img = Input(shape=(self.img_dim,))

        model = Sequential()
        model.add(Dense(512, input_dim=self.img_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1))

        validity = model(img)

        return Model(img, validity)

    def sample_generator_input(self, X, batch_size):
        # Sample random batch of images from X
        idx = np.random.randint(0, X.shape[0], batch_size)
        return X[idx]

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        # Domain A and B (rotated)
        X_A = X_train[:int(X_train.shape[0]/2)]
        X_B = scipy.ndimage.interpolation.rotate(X_train[int(X_train.shape[0]/2):], 90, axes=(1, 2))

        X_A = X_A.reshape(X_A.shape[0], self.img_dim)
        X_B = X_B.reshape(X_B.shape[0], self.img_dim)

        clip_value = 0.01
        n_critic = 4

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):

            # Train the discriminator for n_critic iterations
            for _ in range(n_critic):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Sample generator inputs
                imgs_A = self.sample_generator_input(X_A, batch_size)
                imgs_B = self.sample_generator_input(X_B, batch_size)

                # Translate images to their opposite domain
                fake_B = self.G_AB.predict(imgs_A)
                fake_A = self.G_BA.predict(imgs_B)

                # Train the discriminators
                D_A_loss_real = self.D_A.train_on_batch(imgs_A, valid)
                D_A_loss_fake = self.D_A.train_on_batch(fake_A, fake)

                D_B_loss_real = self.D_B.train_on_batch(imgs_B, valid)
                D_B_loss_fake = self.D_B.train_on_batch(fake_B, fake)

                D_A_loss = 0.5 * np.add(D_A_loss_real, D_A_loss_fake)
                D_B_loss = 0.5 * np.add(D_B_loss_real, D_B_loss_fake)

                # Clip discriminator weights
                for d in [self.D_A, self.D_B]:
                    for l in d.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                        l.set_weights(weights)

            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, imgs_A, imgs_B])

            # Plot the progress
            print ("%d [D1 loss: %f] [D2 loss: %f] [G loss: %f]" \
                % (epoch, D_A_loss[0], D_B_loss[0], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_imgs(epoch, X_A, X_B)

    def save_imgs(self, epoch, X_A, X_B):
        r, c = 4, 4

        # Sample generator inputs
        imgs_A = self.sample_generator_input(X_A, c)
        imgs_B = self.sample_generator_input(X_B, c)

        # Images translated to their opposite domain
        fake_B = self.G_AB.predict(imgs_A)
        fake_A = self.G_BA.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_A, fake_B, imgs_B, fake_A])
        gen_imgs = gen_imgs.reshape((r, c, self.img_rows, self.img_cols, 1))

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[i, j, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = DUALGAN()
    gan.train(epochs=5000, batch_size=32, sample_interval=200)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D1 loss: 0.000076] [D2 loss: 0.000082] [G loss: 175.705750]


/home/critin/miniconda3/envs/ml_werl/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D1 loss: 0.000089] [D2 loss: 0.000092] [G loss: 175.735321]
2 [D1 loss: 0.000092] [D2 loss: 0.000093] [G loss: 176.505493]
3 [D1 loss: 0.000090] [D2 loss: 0.000091] [G loss: 173.956802]
4 [D1 loss: 0.000088] [D2 loss: 0.000089] [G loss: 174.393143]
5 [D1 loss: 0.000085] [D2 loss: 0.000085] [G loss: 171.277588]
6 [D1 loss: 0.000081] [D2 loss: 0.000081] [G loss: 173.835785]
7 [D1 loss: 0.000077] [D2 loss: 0.000077] [G loss: 174.490234]
8 [D1 loss: 0.000073] [D2 loss: 0.000073] [G loss: 169.847076]
9 [D1 loss: 0.000069] [D2 loss: 0.000069] [G loss: 166.729279]
10 [D1 loss: 0.000066] [D2 loss: 0.000065] [G loss: 167.269623]
11 [D1 loss: 0.000062] [D2 loss: 0.000062] [G loss: 163.963394]
12 [D1 loss: 0.000059] [D2 loss: 0.000059] [G loss: 163.993561]
13 [D1 loss: 0.000057] [D2 loss: 0.000056] [G loss: 159.229492]
14 [D1 loss: 0.000055] [D2 loss: 0.000054] [G loss: 158.871643]
15 [D1 loss: 0.000053] [D2 loss: 0.000052] [G loss: 158.106171]
16 [D1 loss: 0.000051] [D2 loss: 0.000051] [G los

...

4461 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.568787]
4462 [D1 loss: 0.000036] [D2 loss: 0.000034] [G loss: 46.634453]
4463 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.585838]
4464 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.718578]
4465 [D1 loss: 0.000039] [D2 loss: 0.000036] [G loss: 47.455063]
4466 [D1 loss: 0.000034] [D2 loss: 0.000039] [G loss: 45.182854]
4467 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.260841]
4468 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.743046]
4469 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.971066]
4470 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.620605]
4471 [D1 loss: 0.000034] [D2 loss: 0.000036] [G loss: 42.376511]
4472 [D1 loss: 0.000036] [D2 loss: 0.000034] [G loss: 43.320717]
4473 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.174366]
4474 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.867493]
4475 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.854095]
4476 [D1 loss: 0.000034] 

4588 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.080421]
4589 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.971638]
4590 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.140717]
4591 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 48.024994]
4592 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.442795]
4593 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.518974]
4594 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.887764]
4595 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 43.480171]
4596 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.771797]
4597 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 46.719578]
4598 [D1 loss: 0.000035] [D2 loss: 0.000035] [G loss: 40.642647]
4599 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.512650]
4600 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.078323]
4601 [D1 loss: 0.000037] [D2 loss: 0.000035] [G loss: 44.939125]
4602 [D1 loss: 0.000034] [D2 loss: 0.000039] [G loss: 43.286919]
4603 [D1 loss: 0.000034] 

4715 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.780945]
4716 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.187279]
4717 [D1 loss: 0.000034] [D2 loss: 0.000033] [G loss: 44.364418]
4718 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.810249]
4719 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.000076]
4720 [D1 loss: 0.000034] [D2 loss: 0.000038] [G loss: 44.759743]
4721 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.316910]
4722 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 44.294960]
4723 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 43.056473]
4724 [D1 loss: 0.000035] [D2 loss: 0.000035] [G loss: 47.018467]
4725 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 46.559532]
4726 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.983070]
4727 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 40.844490]
4728 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 44.546730]
4729 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 44.519836]
4730 [D1 loss: 0.000034] 

4842 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.870880]
4843 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 45.407227]
4844 [D1 loss: 0.000038] [D2 loss: 0.000034] [G loss: 41.456520]
4845 [D1 loss: 0.000034] [D2 loss: 0.000036] [G loss: 43.493507]
4846 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 43.428558]
4847 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 46.036125]
4848 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 45.056679]
4849 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 43.853050]
4850 [D1 loss: 0.000035] [D2 loss: 0.000035] [G loss: 42.945248]
4851 [D1 loss: 0.000034] [D2 loss: 0.000039] [G loss: 46.534599]
4852 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.626492]
4853 [D1 loss: 0.000037] [D2 loss: 0.000034] [G loss: 43.081673]
4854 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.486748]
4855 [D1 loss: 0.000035] [D2 loss: 0.000034] [G loss: 43.472481]
4856 [D1 loss: 0.000037] [D2 loss: 0.000035] [G loss: 45.552391]
4857 [D1 loss: 0.000037] 

4969 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.305180]
4970 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 44.771671]
4971 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.955307]
4972 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.419952]
4973 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 42.857445]
4974 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.735168]
4975 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 41.677128]
4976 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.314911]
4977 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.156052]
4978 [D1 loss: 0.000035] [D2 loss: 0.000035] [G loss: 47.880844]
4979 [D1 loss: 0.000035] [D2 loss: 0.000035] [G loss: 41.063244]
4980 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 43.972076]
4981 [D1 loss: 0.000034] [D2 loss: 0.000035] [G loss: 42.926975]
4982 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 40.325176]
4983 [D1 loss: 0.000034] [D2 loss: 0.000034] [G loss: 47.487732]
4984 [D1 loss: 0.000034] 